In [1]:
#线性回归简洁实现
import torch
import numpy as np
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)


In [2]:
#pytorch提供data包来读取数据
import torch.utils.data as Data

batch_size=10
#将训练数据的特征和标签组合
dataset=Data.TensorDataset(features,labels)
#随机读取小批量
data_iter=Data.DataLoader(dataset,batch_size,shuffle=True)


In [3]:
for X,y in data_iter:
    print(X,y)
    break

tensor([[-0.6403, -0.3721],
        [-0.1560, -0.5522],
        [ 0.2344,  0.3521],
        [ 0.3034, -0.7525],
        [ 1.0518,  0.6406],
        [-0.0278, -1.4302],
        [-0.1689, -0.1168],
        [ 0.9136, -0.6882],
        [ 0.5192,  1.0466],
        [ 0.5222,  1.1927]]) tensor([4.1721, 5.7795, 3.4662, 7.3638, 4.1241, 9.0224, 4.2584, 8.3639, 1.6802,
        1.2006])


In [4]:
#定义模型，使用pytorch预定义的层
from torch import nn
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构


LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [5]:
#也可以用nn.Sequential更方便地搭建网络，Sequantial是一个有序容器，网络层按照传入Sequantial的顺序依次添加
net=nn.Sequential(nn.Linear(num_inputs,1))
#或者：
net=nn.Sequential()
net.add_module('linear',nn.Linear(num_inputs,1))
#net.add_module。。。
#第三种写法：
from collections import OrderedDict
net=nn.Sequential(OrderedDict([('linear',nn.Linear(num_inputs,1))]))
print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [6]:
#通过net.parameters来查看模型所有可学习的参数
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.6801, -0.5213]], requires_grad=True)
Parameter containing:
tensor([-0.4130], requires_grad=True)


In [7]:
#注意torch.nn仅支持输入一个batch的样本，不支持单个样本的输入，如果只有单个样本，用input.unsqueeze(0)来添加一维

#初始化模型参数，pytorch在init模块中提供了多种初始化方法，init是initializer的缩写形式。通过init.normal_把权重参数初始化为N(0,0.01^2)的正态，偏差初始化为0
from torch.nn import init
init.normal_(net[0].weight,mean=0,std=0.01)
init.constant_(net[0].bias,val=0)


Parameter containing:
tensor([0.], requires_grad=True)

In [8]:
#定义损失函数，直接用nn模块提供的
loss=nn.MSELoss()

In [9]:
#定义优化算法，torch.optim提供常用优化算法
import torch.optim as optim
optimizer=optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [10]:
#可以为不同子网络设置不同的学习率
optimizer =optim.SGD([
                # 如果对某个参数不指定学习率，就使用最外层的默认学习率
                {'params': net.subnet1.parameters()}, # lr=0.03
                {'params': net.subnet2.parameters(), 'lr': 0.01}
            ], lr=0.03)


AttributeError: 'Sequential' object has no attribute 'subnet1'

In [ ]:
#调整学习率
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍


In [11]:
#训练模型
num_epochs=3
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        output=net(X)
        l=loss(output,y.view(-1,1))
        optimizer.zero_grad()#梯度清零，等价于net.zero_grad
        l.backward()
        optimizer.step()#迭代模型参数，按小批量随机梯度下降
    print('epoch %d,loss: %f'%(epoch,l.item()))

epoch 1,loss: 0.000219
epoch 2,loss: 0.000149
epoch 3,loss: 0.000144


In [12]:
dense=net[0]
print(true_w,dense.weight)
print(true_b,dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9998, -3.3998]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2003], requires_grad=True)
